In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("sp500_testdata_2.csv")

In [3]:
df.Date = pd.to_datetime(df.Date)
df["TR"] = ""
df.TR.loc[0] = df.high.loc[0] - df.low.loc[0]    # Initial value for TR
df.TR.loc[1:] = ""
df["N"] = ""
df["N"].loc[0] = df.high.loc[0] - df.low.loc[0]   # Initial value for N
df["N"].loc[1:] = ""
df["NPP"] = ""
df.NPP.loc[0] = 0.00     # Initial value for NPP
df["c_type"] = ""
df["FL"] = ""
df["SL"] = ""
df.FL.loc[0] = 0.00      # Initial value for FL
df.SL.loc[0] = 0.00      # Initial value for SL
df["Sep"] = ""           # Percentage separation between FL and SL
df.Sep.loc[0] = 0.00
df["TrendDir"] = ""
df["TrendDir"] = df["TrendDir"].astype('str', errors='ignore')
df["TrendDay"] = ""
df["TrendDay"] = df["TrendDay"].astype('int', errors='ignore')
df["FE_FL"] = ""       
df["AE_FL"] = ""
df["SP"] = ""
df["En_price1"] = ""
df["Position"] = ""
df["Counter1"] = ""
df["Counter1"] = df["Counter1"].astype('int', errors='ignore')
df["Counter2"] = ""
df["Counter2"] = df["Counter2"].astype('int', errors='ignore')
df["SDEE"] = ""   # same day entry/exit: Position=0 or 1 and trade entry/exit occurs on same day due to SL (a & b) or TP (c) or RE (d)     
df["a"] = ""
df["b"] = ""
df["c"] = ""
df["d"] = ""
df["RE_CAN"] = ""
df["RE_CAN"] = df["RE_CAN"].astype('bool', errors='ignore')
# df["EnP2_RE"] = ""
df["En_price2"] = ""
df["Ex_price1"] = ""
df["Ex_price2"] = ""
df["Ex_price3"] = ""
df["Ex_price4"] = ""
df["Ex_price5"] = ""
df["P_L"] = ""
df["Capital"] = ""
df["Capital"].loc[0:25] = 25000

df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]].apply(pd.to_numeric)



C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df.dtypes

Date         datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
TR                  float64
N                   float64
NPP                 float64
c_type               object
FL                  float64
SL                  float64
Sep                 float64
TrendDir             object
TrendDay            float64
FE_FL               float64
AE_FL               float64
SP                  float64
En_price1           float64
Position            float64
Counter1             object
Counter2             object
SDEE                 object
a                   float64
b                   float64
c                   float64
d                   float64
RE_CAN                 bool
En_price2           float64
Ex_price1           float64
Ex_price2           float64
Ex_price3           float64
Ex_price4           float64
Ex_price5           float64
P_L                 float64
Capital             float64
dtype: object

In [5]:
#  4 loops: 
#   a) calculate True Range for every row after the first row
#   b) calculate N for every row after the first row
#   c) calculate NPP for every row after first row
#   d) determine Candle type: open, dark, doji_open, doji_dark


rownum = 1
for rownum in range(1,1500):
    
    """ a)  True Range """
    #  True Range = max( absolute value(H - L), absolute value (H - C(previous)), absoulute value (C(previous) - L))
    df.TR.loc[rownum:] = max(abs(df.high.loc[rownum] - df.low.loc[rownum]), abs(df.high.loc[rownum] - df.close.loc[rownum-1]), abs(df.close.loc[rownum-1] - df.low.loc[rownum]))
    
    """ b)  N  """
    #  N = N(previous) + (TR-N(previous)) / (0.5 * 20 + 1)
    Constant = 20
    df.N.loc[rownum:] = df.N.loc[rownum - 1]+(df.TR.loc[rownum]-df.N.loc[rownum - 1])/(0.5 * (Constant + 1))
    
    """ c)  NPP  """
    df.NPP.loc[rownum:] = df.N.loc[rownum]/df.close.loc[rownum-1] * 100
    
    """ d)  c_type (candle type)  """
    if df.close.loc[rownum] > df.open.loc[rownum]:
        df.c_type.loc[rownum:] = "o"
    elif df.close.loc[rownum] < df.open.loc[rownum]:
        df.c_type.loc[rownum:] = "d"
    elif df.close.loc[rownum] == df.open.loc[rownum]:
        candle_range = df.high.loc[rownum] - df.low.loc[rownum]
        if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
            df.c_type.loc[rownum:] = "d-o"
        else:
            df.c_type.loc[rownum:] = "d-d"    # open & close occur in lower half of range 
            
    rownum += 1
    
    
"""    
#  b)
#  N = N(previous) + (TR-N(previous)) / (0.5 * 20 + 1)
Constant = 20
rownum = 1
for rownum in range(1, 1500):
    df.N.loc[rownum:] = df.N.loc[rownum - 1]+(df.TR.loc[rownum]-df.N.loc[rownum - 1])/(0.5 * (Constant + 1))
    rownum +=1
    
# c)
df.NPP.loc[rownum:].round(3)
rownum = 1
for rownum in range(1, 1500):
    df.NPP.loc[rownum:] = df.N.loc[rownum]/df.close.loc[rownum-1] * 100
    rownum += 1
    
# d)

df.c_type.loc[rownum]
    
    
# try replacing this with a List Comprehension    
# [df.NPP.loc[rownum:] for df.NPP.loc[rownum] in range(1.599) if df.NPP.loc[rownum] is float]

"""

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


'    \n#  b)\n#  N = N(previous) + (TR-N(previous)) / (0.5 * 20 + 1)\nConstant = 20\nrownum = 1\nfor rownum in range(1, 1500):\n    df.N.loc[rownum:] = df.N.loc[rownum - 1]+(df.TR.loc[rownum]-df.N.loc[rownum - 1])/(0.5 * (Constant + 1))\n    rownum +=1\n    \n# c)\ndf.NPP.loc[rownum:].round(3)\nrownum = 1\nfor rownum in range(1, 1500):\n    df.NPP.loc[rownum:] = df.N.loc[rownum]/df.close.loc[rownum-1] * 100\n    rownum += 1\n    \n# d)\n\ndf.c_type.loc[rownum]\n    \n    \n# try replacing this with a List Comprehension    \n# [df.NPP.loc[rownum:] for df.NPP.loc[rownum] in range(1.599) if df.NPP.loc[rownum] is float]\n\n'

In [6]:
# 3 min 42 sec to finish execution on 1500 rows
# loop that yields FL, SL, Sep values after index position zero

rownum = 1
FL_period = 7
SL_period = 20
for rownum in range(1, 1500):
    df.FL.loc[rownum:] = df.FL.loc[rownum-1] + (df.close.loc[rownum] - df.FL.loc[rownum-1])/((FL_period+1)/2)
    df.SL.loc[rownum:] = df.SL.loc[rownum-1] + (df.close.loc[rownum] - df.SL.loc[rownum-1])/((SL_period+1)/2)
    df.Sep.loc[rownum] = (df.FL.loc[rownum] - df.SL.loc[rownum])/df.SL.loc[rownum] * 100
    rownum += 1
    
    
    
# Determining TrendDir based on boolean result for question: FL > SL ?
# start at index = 25 to give ramp-up time for lags
rownum = 100
for rownum in range(25, 1500):
    if df.FL.loc[rownum] > df.SL.loc[rownum]:
        df.TrendDir.loc[rownum] = "Up"
    else:    
        df.TrendDir.loc[rownum] = "Down"
rownum += 1

# iterate through rows starting at row 101 to find where trends initiate (TrendDay=0) and continue (TrendDay>0)

rownum = 101           # start at index = 101 (TrendDay at index=100 is set to 0)
for rownum in range(25, 1500):
    if df.TrendDir.loc[rownum] == df.TrendDir.loc[rownum-1]:
        df.TrendDay.loc[rownum] = (df.TrendDay.loc[rownum-1] + 1)
    else:
        df.TrendDay.loc[rownum] = 0
rownum += 1


C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# 1 min 23 sec to finish execution on 1500 rows

""" FE_FL and AE_FL - favorable & adverse distance of a day's High & Low from the previous day's FL in terms of previous N """

rownum = 25   # start after series for TrendDay transitions from blank (string value) to int
for rownum in range(25, 1500):
    if df.TrendDay.loc[rownum] == 0:        
        df.FE_FL.loc[rownum] = ""
        df.AE_FL.loc[rownum] = ""
    else:
        if df.TrendDir.loc[rownum] == "Up":
        
            #Favorable Excursion from FL in terms of N for long position:
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.high.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for long position (display positive value):
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.low.loc[rownum] - df.FL.loc[rownum-1])/df.N.loc[rownum-1])
    
        elif df.TrendDir.loc[rownum] == "Down":
        
            #Favorable Excursion from FL in terms of N for Short position:
            if df.low.loc[rownum] > df.FL.loc[rownum-1]:
                df.FE_FL.loc[rownum] = 0
            else:
                df.FE_FL.loc[rownum] = (df.FL.loc[rownum-1] - df.low.loc[rownum])/df.N.loc[rownum-1]

            #Adverse Excursion from FL in terms of N for for Short position (display positive value):
            if df.high.loc[rownum] < df.FL.loc[rownum-1]:
                df.AE_FL.loc[rownum] = 0
            else:
                df.AE_FL.loc[rownum] = abs((df.FL.loc[rownum-1] - df.high.loc[rownum])/df.N.loc[rownum-1])        
            
# incrememnt to next bar
rownum += 1

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
type(df.Capital.loc[50])

In [8]:
"""  calculates and displays in Dataframe A,B,C volatility levels """


addpos = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},                            # syntax to call, enter script as follows:
          "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}                             # addpos["L"]["2"]  yields: 0.015
          }                                                                        # stoploss["S"]["3"]  yields: 0.0195

stoploss = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

takeprof = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

re_entry = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

X = 0.25   # General Risk Mgmt. multiplier

rownum = 10
for rownum in range (1, 1500):
    if df.TrendDay.loc[rownum] > 0 and df.TrendDir.loc[rownum] == "Up":

        """ AP_U function: AE_FL: price at which trigger AP """
        if df.NPP.loc[rownum - 1] < addpos["L"]["1"]:
            a = df.FL.loc[rownum - 1] +  X * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["2"]:                    
            a = df.FL.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["L"]["3"]:
            a = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]
        else:
            a = df.FL.loc[rownum - 1] - 6.4 * X * df.N.loc[rownum - 1]

        df.a.loc[rownum] = a   # print value in dataframe slice 

        """ SL_U function: AE_FL: price at which triggers SL """
        if df.NPP.loc[rownum - 1] <= stoploss["L"]["1"]:
            b =  df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N below FL
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["2"]:
            b = df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N below FL
        elif df.NPP.loc[rownum - 1] <= stoploss["L"]["3"]:
            b = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N below FL
        else:
            b =  df.FL.loc[rownum - 1] - 8 * X * df.N.loc[rownum - 1]

        df.b.loc[rownum] = b

        """ TP_U function: FE_FL: price at which triggers TP """
        if df.NPP.loc[rownum - 1] <= takeprof["L"]["1"]:
            c = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["2"]:
            c = df.FL.loc[rownum - 1] + 6.5 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 163%N (6.5 * X) above previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["L"]["3"]:
            c = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) above previous FL
        else:
            c = df.FL.loc[rownum - 1] + 7.5 * X * df.N.loc[rownum - 1]                    

        df.c.loc[rownum] = c

        rownum += 1
        
    elif df.TrendDay.loc[rownum] > 0 and df.TrendDir.loc[rownum] == "Down":
        
        """ AP_D function: AE_FL: price at which triggers AP """
        if df.NPP.loc[rownum - 1] < addpos["S"]["1"]:
            a = df.FL.loc[rownum - 1] -  X * df.N.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["S"]["2"]:                    
            a = df.FL.loc[rownum - 1]
        elif df.NPP.loc[rownum - 1] <= addpos["S"]["3"]:
            a = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]
        else:
            a = df.FL.loc[rownum - 1] + 6.4 * X * df.N.loc[rownum - 1]

        df.a.loc[rownum] = a   # print value in dataframe slice  
        
        """ SL_D function: AE_FL: price at which trigger SL """
        if df.NPP.loc[rownum - 1] <= stoploss["S"]["1"]:
            b =  df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N above FL
        elif df.NPP.loc[rownum - 1] <= stoploss["S"]["2"]:
            b = df.FL.loc[rownum - 1] + 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N above FL
        elif df.NPP.loc[rownum - 1] <= stoploss["S"]["3"]:
            b = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N above FL
        else:
            b =  df.FL.loc[rownum - 1] + 8 * X * df.N.loc[rownum - 1]

        df.b.loc[rownum] = b        
        
        
        """ TP_D function: FE_FL: price at which triggers TP """
        if df.NPP.loc[rownum - 1] <= takeprof["S"]["1"]:
            c = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N below previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["S"]["2"]:
            c = df.FL.loc[rownum - 1] - 6.5 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 163%N (6.5 * X) below previous FL
        elif df.NPP.loc[rownum - 1] <= takeprof["S"]["3"]:
            c = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) below previous FL
        else:
            c = df.FL.loc[rownum - 1] - 7.5 * X * df.N.loc[rownum - 1]                    

        df.c.loc[rownum] = c

        rownum += 1
                
        


C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Counter1", "Counter2", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Counter1", "Counter2", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]].apply(pd.to_numeric)

In [53]:
df["SP"].fillna("", inplace = True)
df["En_price1"].fillna("", inplace = True)
df["Position"].fillna("", inplace = True)
df["Counter1"].fillna("", inplace = True)
df["Counter2"].fillna("", inplace = True)
df["SDEE"].fillna("", inplace = True)
df["a"].fillna("", inplace = True)
df["b"].fillna("", inplace = True)
df["c"].fillna("", inplace = True)
df["d"].fillna("", inplace = True)
df["RE_CAN"].fillna("", inplace = True)
df["En_price2"].fillna("", inplace = True)
df["Ex_price1"].fillna("", inplace = True)
df["Ex_price2"].fillna("", inplace = True)
df["Ex_price3"].fillna("", inplace = True)
df["Ex_price4"].fillna("", inplace = True)
df["Ex_price5"].fillna("", inplace = True)
df["P_L"].fillna("", inplace = True)
df["Capital"].fillna("", inplace = True)


In [56]:
sigfigs = {
            'N': 2, 'NPP': 3, 'FL': 2, 'SL': 2, 'Sep': 3, 'FE_FL': 3,
           'AE_FL': 3, 'TrendDay': 0, 'SP': 2, 'a':0, 'b':0, 'c':0
          }

df.iloc[60:120, 2:22].round(sigfigs)

,high,low,close,TR,N,NPP,c_type,FL,SL,Sep,TrendDir,TrendDay,FE_FL,AE_FL,SP,En_price1,Position,Counter1,Counter2,SDEE
60,1150.00,1127.75,1145.75,22.25,34.97,3.073,o,1151.49,1165.11,-1.169,Down,21.0,0.707,0.000,,,,,,
61,1171.00,1140.25,1157.25,30.75,34.56,3.017,o,1152.93,1164.36,-0.982,Down,22.0,0.321,0.558,,,,,,
62,1178.00,1151.00,1171.25,27.00,33.84,2.924,o,1157.51,1165.02,-0.644,Down,23.0,0.056,0.725,,,,,,
63,1179.00,1154.75,1158.50,24.25,32.93,2.812,d,1157.76,1164.40,-0.570,Down,24.0,0.082,0.635,,,,,,
64,1164.75,1144.50,1158.75,20.25,31.72,2.738,o,1158.01,1163.86,-0.503,Down,25.0,0.403,0.212,,,,,,
65,1176.00,1154.00,1175.50,22.00,30.80,2.658,o,1162.38,1164.97,-0.222,Down,26.0,0.126,0.567,,,,,,
66,1177.25,1152.75,1163.00,24.50,30.20,2.569,o,1162.53,1164.78,-0.193,Down,27.0,0.313,0.483,,,,,,
67,1172.25,1140.25,1166.50,32.00,30.37,2.611,o,1163.53,1164.94,-0.122,Down,28.0,0.738,0.322,,,,,,
68,1176.75,1146.75,1154.50,30.00,30.33,2.600,d,1161.27,1163.95,-0.230,Down,29.0,0.552,0.435,,,,,,
69,1165.00,1134.50,1163.00,30.50,30.35,2.629,o,1161.70,1163.86,-0.185,Down,30.0,0.883,0.123,,,,,,


In [26]:
""" Rote functions """

""" Uptrend, Pos=1      c_type: open & d-o       case: high>c, low<a&b  """
def _1f():
    En_price2 = df.a.loc[rownum-1] - tick
    Ex_price3 = df.b.loc[rownum - 1] - tick
    Ex_price4 = df.b.loc[rownum - 1] - tick
    df.Counter1.loc[rownum] = ""
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position
    P_L = ((Ex_price3 - En_price1) + (Ex_price4 - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1
    
""" Uptrend, Pos=1      c_type: open & d-o       case: high>c, low<a   """    
def _1g():
    En_price = df.a.loc[rownum-1] - tick
    Ex_price1 = df.c.loc[rownum - 1] + tick
    Ex_price2 = df.c.loc[rownum - 1] + tick
    df.Counter1.loc[rownum] = ""
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position
    RE_CAN = True
    df.RE_CAN.loc[rownum] = RE_CAN
    P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1
    
""" Uptrend, Pos=1      c_type: open & d-o       case: high>c, low>a    """    
def _1h():
    Ex_price1 = df.c.loc[rownum - 1] + tick    
    df.Counter1.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position
    RE_CAN = True
    df.RE_CAN.loc[rownum] = RE_CAN
    P_L = (Ex_price1 - En_price1) / tick * tick_val
    df.P_L.loc[rownum] = P_L  
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1    

""" Uptrend, Pos=1      c_type: open & d-o       case: high<c, low<a&b   """    
def _1j():
    _1f()    # simply call function _1f()
    
""" Uptrend, Pos=1      c_type: open & d-o       case: high<c, low<a     """    
def _1k():
    En_price2 = df.a.loc[rownum-1] - tick
    df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
    df.Counter2.loc[rownum] = df.Counter2.loc[rownum] + 1
    Position = 2
    df.Position.loc[rownum] = Position
    P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]   # line 57 error?
    rownum += 1

""" Uptrend, Pos=1      c_type: open & d-o       case: high<c, low>a     """  
def _1m():
    df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
    df.Position.loc[rownum] = df.Position.loc[rownum-1]
    P_L = (df.close.loc[rownum] - En_price1) / tick * tick_val       
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1       
           
""" Uptrend, Pos=1      c_type: DARK & d-D       case: high>c, low<a&b    """           
def _1n():
    Ex_price1 = df.c.loc[rownum-1] + tick
    df.Counter1.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position
    P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L    
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1
           
""" Uptrend, Pos=1      c_type: DARK & d-D       case: high>c, low<a      """             
def _1p():
    _1n()            # simply call function def _1n()
           
           
""" Uptrend, Pos=1      c_type: DARK & d-D       case: high>c, low>a      """             
def _1q():
    _1n()            # simply call function def _1n()           
           

""" Uptrend, Pos=1      c_type: DARK & d-D       case: high<c, low<a&b       """                
def _1r():
    _1f()            # simply call function def _1f()
           

""" Uptrend, Pos=1      c_type: DARK & d-D       case: high<c, low<a        """                
def _1s():
    _1k()            # simply call function def _1k() 
           
           
""" Uptrend, Pos=1      c_type: DARK & d-D       case: high<c, low>a        """                
def _1t():
    _1m()            # simply call function def _1m()           
           
           
""" Uptrend, Pos=2     c_type: open & d-o       case: high>c  low <a&b     """           
def _2f():
    Ex_price3 = df.b.loc[rownum] - tick
    Ex_price4 = df.b.loc[rownum] - tick
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position           
    P_L = ((Ex_price3 - En_price1) + (Ex_price4 - En_price2)) / tick * tick_val       
    df.P_L.loc[rownum] = P_L    
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1           
           

""" Uptrend, Pos=2     c_type: open & d-o       case: high>c  low <a&b     """           
def _2g():   
    Ex_price1 = c + tick
    Ex_price2 = c + tick       
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position           
    P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val       
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1                      
           
""" Uptrend, Pos=2     c_type: open & d-o       case: high>c  low >a      """           
def _2h():              
    _2g()       # simply call function def _1g()
           
           
""" Uptrend, Pos=2     c_type: open & d-o       case: high<c    low < a&b      """           
def _2j():                         
    Ex_price3 = b - tick
    Ex_price4 = b - tick
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position           
    P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val       
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1               
           
           
""" Uptrend, Pos=2     c_type: open & d-o       case: high<c    low < a       """           
def _2k():                                    
    df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
    df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1     
    P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val       
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1       
           
           
""" Uptrend, Pos=2     c_type: open & d-o       case: high < c      low > a       """           
def _2m():
    _2k()       
           
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high > c    low < a&b       """           
def _2n():           
    _2g()       
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high > c    low < a         """           
def _2p():               
    _2g()       
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high > c    low > a         """           
def _2q():               
    _2g()             
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high < c    low < a&b         """           
def _2r():                          
    _2j()       
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high < c    low < a           """           
def _2s():            
    _2k()
           
           
""" Uptrend, Pos=2     c_type: DARK & d-d       case: high < c    low > a           """           
def _2t():            
    _2k()           
           
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d & b           """           
def _0f():              
    En_En_price1 = d - tick
    En_price2 = d - tick
    Ex_price3 = b - tick
    Ex_price4 = b - tick
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position 
    P_L = ((Ex_price3- En_price1) + (Ex_price4 - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1
  
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d            """           
def _0g():
    En_price1 = d - tick
    En_price2 = d - tick           
    Ex_price1 = c + tick
    Ex_price2 = c + tick
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position 
    P_L = ((Ex_price1- En_price1) + (Ex_price2 - En_price2)) / tick * tick_val
    df.P_L.loc[rownum] = P_L
    df.Capital.loc[rownum] = Capital.loc[rownum-1] + df.P_L.loc[rownum]
    rownum += 1
    
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high > c    low > d            """           
def _0h():           
    df.Counter1.loc[rownum] = ""  
    df.Counter2.loc[rownum] = ""
    Position = 0
    df.Position.loc[rownum] = Position 
    df.Capital.loc[rownum] = df.Capital.loc[rownum-1]
    rownum += 1
           
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d&b            """           
def _0j():
    _0f()       
           
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d               """           
def _0k():           
    En_price1 = d - tick
    En_price2 = d - tick              
    df.Counter1.loc[rownum] = 1
    df.Counter2.loc[rownum] = 1
           

""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low > d               """           
def _0m():            
    _0h()       
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high > c    low < d&b               """           
def _0n():            
    _0f()                
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high > c    low < d                """           
def _0o():            
    _0k()           
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high > c    low > d                """           
def _0q():            
    _0h()            
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d&b                """           
def _0r():            
    _0f()
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low < d                  """           
def _0s():            
    _0k()           
           
""" Uptrend, Pos=0     c_type: open & d-o       case: high < c    low > d                  """           
def _0t():            
    _0h()            
           
           
           
           

In [51]:
df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Counter1", "Counter2", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Counter1", "Counter2", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]].apply(pd.to_numeric)

In [50]:
"""  Identifies Long & Short entries:  sets and displays in Dataframe: En_price1, Position = 1, Counter1 = 1  """

tick = 0.25
tick_val = 12.50
Capital = float(25000)
df["Capital"] = pd.to_numeric(df["Capital"], errors = "coerce")

rownum = 50                                                 # start looking for uptrend starts from row 50


for rownum in range(50, 1499):
            
    if df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":          
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.high.loc[rownum]                            # assign the 'high' slice to the SP variable
        df.SP.loc[rownum] = SP                              # equate the SP slice with the SP variable
        rownum += 1
        
        while df.TrendDay.loc[rownum] > 0:              # this loop finds initial entry, records initial P_L, then advances to next day
            if df.high.loc[rownum] > SP:
                En_price1 = SP + tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = 1
                Position = 1                                
                df.Position.loc[rownum] = Position         
                df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                rownum += 1
                break

                                  
            else:
                SP = df.high.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1  

      
    elif df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Down":
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.low.loc[rownum]
        df.SP.loc[rownum] = SP
        rownum += 1        

        while df.TrendDay.loc[rownum] > 0:
            if df.low.loc[rownum] < SP:
                En_price1 = SP - tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = Counter1
                Position = 1
                df.Position.loc[rownum] = Position
                df.P_L.loc[rownum] = (En_price1 - df.close.loc[rownum]) / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]                
                rownum += 1
                break
            else:
                SP = df.low.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1    

                


                               
         

C:\Users\tony685\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
df["Capital"] = pd.to_numeric(df["Capital"], errors = "coerce")
type(df.Capital.loc[rownum])

numpy.float64

In [44]:
rownum = 50
for rownum in range(50, 1500):


    """ begin reacting to intratrend volatility """

    while df.TrendDay.loc[rownum] >= 1:
        if df.Position.loc[rownum-1] == 1 and df.TrendDir.loc[rownum] == "Up":
            if df.c_type.loc[rownum] == "o" or df.c_type.loc[rownum] == "d-o":

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:      # case 1f
                        _1f()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 1g
                        _1g()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:     # case 1h
                        Ex_price1 = df.c.loc[rownum - 1] + tick    
                        df.Counter1.loc[rownum] = ""
                        Position = 0
                        df.Position.loc[rownum] = Position
                        RE_CAN = True
                        df.RE_CAN.loc[rownum] = RE_CAN
                        P_L = (Ex_price1 - En_price1) / tick * tick_val
                        df.P_L.loc[rownum] = P_L  
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1    

                elif df.high.loc[rownum] < df.c.loc[rownum]:         
                    if df.low.loc[rownum] < df.b.loc[rownum]:       # case 1j
                        _1j()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 1k
                        En_price2 = df.a.loc[rownum-1] - tick
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum] + 1
                        Position = 2
                        df.Position.loc[rownum] = Position
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]   # line 57 error?
                        rownum += 1

                    elif df.low.loc[rownum] > df.a.loc[rownum]:      # case 1m
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Position.loc[rownum] = df.Position.loc[rownum-1]
                        P_L = (df.close.loc[rownum] - En_price1) / tick * tick_val       
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1

            elif df.c_type.loc[rownum] == "d" or df.c_type.loc[rownum] == "d-d":   

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 1n
                        _1n()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:   # case 1p
                        _1p()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 1q    
                        _1q()

                elif df.high.loc[rownum] < df.c.loc[rownum]:
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 1r
                        _1r()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:  # case 1s
                        En_price2 = df.a.loc[rownum-1] - tick
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum] + 1
                        Position = 2
                        df.Position.loc[rownum] = Position
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]   # line 57 error?
                        rownum += 1
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 1t
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Position.loc[rownum] = df.Position.loc[rownum-1]
                        P_L = (df.close.loc[rownum] - En_price1) / tick * tick_val       
                        df.P_L.loc[rownum] = P
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1


        elif df.Position.loc[rownum-1] == 2 and df.TrendDir.loc[rownum] == "Up":

            if df.c_type.loc[rownum] == "o" or df.c_type.loc[rownum] == "d-o":

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:      # case 2f
                        _2f()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 2g
                        _2g()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:     # case 2h
                        _2h()

                elif df.high.loc[rownum] < df.c.loc[rownum]:         
                    if df.low.loc[rownum] < df.b.loc[rownum]:       # case 2j
                        _2j()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 2k
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1     
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val       
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1       
                    elif df.low.loc[rownum] > df.a.loc[rownum]:      # case 2m
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1     
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val       
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1       

        elif df.c_type.loc[rownum] == "d" or df.c_type.loc[rownum] == "d-d":   

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 2n
                        _2n()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:   # case 2p
                        _2p()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 2q    
                        _2q()

                elif df.high.loc[rownum] < df.c.loc[rownum]:
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 2r
                        _2r()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:  # case 2s
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1     
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val       
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1       
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 2t
                        df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                        df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1     
                        P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val       
                        df.P_L.loc[rownum] = P_L
                        df.Capital.loc[rownum] = df.Capital.loc[rownum-1] + df.P_L.loc[rownum]
                        rownum += 1       

                
                
                
                
                    
                    
                    
                    
                    
                    
                    

KeyError: 'the label [50] is not in the [index]'

In [24]:
rownum = 10
for rownum in range(1, 1500):
    while df.TrendDay.loc[rownum] >= 1:
        
        if df.Position.loc[rownum] == 1 and df.TrendDir.loc[rownum] == "Up":

            if df.c_type.loc[rownum] == "o" or df.c_type.loc[rownum] == "d-o":

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:      # case 1f
                        _1f()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 1g
                        _1g()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:     # case 1h
                        _1h()

                elif df.high.loc[rownum] < df.c.loc[rownum]:         
                    if df.low.loc[rownum] < df.b.loc[rownum]:       # case 1j
                        _1j()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 1k
                        _1k()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:      # case 1m
                        _1m()

            elif df.c_type.loc[rownum] == "d" or df.c_type.loc[rownum] == "d-d":   

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 1n
                        _1n()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:   # case 1p
                        _1p()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 1q    
                        _1q()

                elif df.high.loc[rownum] < df.c.loc[rownum]:
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 1r
                        _1r()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:  # case 1s
                        _1s()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 1t
                        _1t()


        elif df.Position.loc[rownum] == 2 and df.TrendDir.loc[rownum] == "Up":

            if df.c_type.loc[rownum] == "o" or df.c_type.loc[rownum] == "d-o":

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:      # case 2f
                        _2f()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 2g
                        _2g()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:     # case 2h
                        _2h()

                elif df.high.loc[rownum] < df.c.loc[rownum]:         
                    if df.low.loc[rownum] < df.b.loc[rownum]:       # case 2j
                        _2j()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:     # case 2k
                        _2k()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:      # case 2m
                        _2m()    

            elif df.c_type.loc[rownum] == "d" or df.c_type.loc[rownum] == "d-d":   

                if df.high.loc[rownum] > df.c.loc[rownum]: 
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 2n
                        _2n()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:   # case 2p
                        _2p()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 2q    
                        _2q()

                elif df.high.loc[rownum] < df.c.loc[rownum]:
                    if df.low.loc[rownum] < df.b.loc[rownum]:    # case 2r
                        _2r()
                    elif df.low.loc[rownum] <= df.a.loc[rownum]:  # case 2s
                        _2s()
                    elif df.low.loc[rownum] > df.a.loc[rownum]:   # case 2t
                        _2t()    

"""        
    elif df.Position.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":
"""       
    
    
    
    
    
    
    
    
    
    
    
    
    
    

KeyboardInterrupt: 

In [ ]:
df.to_csv("codestudy CSV FILE__1.csv")

In [ ]:
rownum = 10
for rownum in range(10, 1499):

    """  Position = 1 in Uptrend  """  

    if df.Position.loc[rownum] == 1 and df.TrendDir.loc[rownum] == "Up": 

        """ Open Candle:  low occurs before high """
        if df.close.loc[rownum] > df.open.loc[rownum]:
        
            """ Case 1: High > TP  """
            if df.high.loc[rownum] > df.c.loc[rownum]: 
                
                """ Case 1.1:  Low < b (SL):  add position then stop out on both  """
                if df.low.loc[rownum] < df.a.loc[rownum] and df.low.loc[rownum] < df.b.loc[rownum]:

                    # Pos 1 ---> Pos 2 ---> Pos 0
                    Position = 0
                    df.Position.loc[rownum] = Position
                    En_price2 = df.a.loc[rownum] - tick
                    Ex_price3 = df.b.loc[rownum] - tick
                    Ex_price4 = df.b.loc[rownum] - tick
                    df.Counter1.loc[rownum] = ""
                    df.Counter2.loc[rownum] = ""
                    P_L = ((Ex_price3 - En_price1) + (Ex_price4 - En_price2)) / tick * tick_val
                    df.P_L.loc[rownum] = P_L
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1
                
                """ Case 1.2:  Low < a (AP) but Low > b (SL):  add position then take profit on both  """
                elif df.low.loc[rownum] < df.a.loc[rownum] and df.low.loc[rownum] > df.b.loc[rownum]:
                    
                    # Pos1 ---> Pos 2 ---> Pos 0
                    Position = 0
                    df.Position.loc[rownum] = Position 
                    En_price2 = df.a.loc[rownum] - tick
                    Ex_price1 = df.c.loc[rownum] + tick
                    Ex_price2 = df.c.loc[rownum] + tick
                    df.Counter1.loc[rownum] = ""
                    df.Counter2.loc[rownum] = ""
                    P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1
                    
                """ Case 1.3:  Low > a (AP) and b (SL):  take profit on initial position and stage for Re-Entry  """
                elif df.low.loc[rownum] > df.a.loc[rownum] and df.low.loc[rownum] > df.b.loc[rownum]:
                    Position = 0
                    df.Position.loc[rownum] = Position
                    Ex_price1 = df.c.loc[rownum] + tick
                    df.Counter1.loc[rownum] = ""
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = RE_CAN
                    P_L = (Ex_price1 - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1                
                
                    
                    
            """ Case 2: High < TP """
            if df.high.loc[rownum] < df.c.loc[rownum]:
                
                """ Case 2.1:  Low < b  : add position and stop out on both  """
                if df.low.loc[rownum] < df.a.loc[rownum] and df.low.loc[rownum] < df.b.loc[rownum]:
                    
                    # Pos 1 ---> Pos 2 ---> Pos 0
                    Position = 0
                    df.Position.loc[rownum] = Position
                    En_price2 = df.a.loc[rownum] - tick
                    Ex_price3 = df.b.loc[rownum] - tick
                    Ex_price4 = df.b.loc[rownum] - tick
                    df.Counter1.loc[rownum] = ""
                    df.Counter2.loc[rownum] = ""
                    P_L = ((Ex_price3 - En_price1) + (Ex_price4 - En_price2)) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1                

                """ Case 2.2:  Low < a (AP) and Low > b (SL):  add position and carry both positions to next day  """
                elif df.low.loc[rownum] < df.a.loc[rownum] and df.low.loc[rownum] > df.b.loc[rownum]:
                    
                    # Pos1 ---> Pos 2 
                    Position = 2
                    df.Position.loc[rownum] = Position 
                    En_price2 = df.a.loc[rownum] - tick
                    df.Counter1.loc[rownum] = df.Counter1.loc[rownum] + 1
                    df.Counter2.loc[rownum] = df.Counter2.loc[rownum] + 1
                    P_L = ((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1
            
            
                    
        """ Dark Candle:  high occurs before low.  Take profit on initial position and stage for Re-Entry  """            
        elif df.close.loc[rownum] < df.open.loc[rownum]:
            
            # Pos 1 ---> Pos 0
            Position = 0
            df.Position.loc[rownum] = Position
            df.Counter1.loc[rownum] = ""
            Ex_price1 = df.c.loc[rownum] + tick
            RE_CAN = True
            df.RE_CAN.loc[rownum] = RE_CAN
            P_L = ((df.close.loc[rownum] - Ex_price1)) / tick * tick_val
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        """ Doji Candle:  open equals close:  identify location of open/close and classify as Open or Dark  """
        elif df.close.loc[rownum] == df.open.loc[rownum]:
            Candle_Range = df.high.loc[rownum] - df.low.loc[rownum]
            if df.open.loc[rownum] >= (df.low.loc[rownum] +  0.5 * Candle_Range):  # open & close occur at middle or in top half of range
                candle = "doji_open"  # treat like candle=OPEN
                Position = 0
                df.Position.loc[rownum] = Position
                En_price2 = df.a.loc[rownum] - tick
                Ex_price1 = df.c.loc[rownum] + tick
                Ex_price2 = df.c.loc[rownum] + tick
                df.Counter1.loc[rownum] = ""
                df.Counter2.loc[rownum] = ""
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                P_L = ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                rownum += 1


            elif df.open.loc[rownum] < (df.low.loc[rownum] + 0.5 * Candle_Range):
                candle = "doji_dark"   # open & close occur in lower half of range:  treat like candle=DARK
                Position = 0
                df.Counter1.loc[rownum] = ""
                Ex_price1 = df.c.loc[rownum] + tick
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                P_L = ((df.close.loc[rownum] - Ex_price1)) / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                rownum += 1

        """ Case 2: High < TP and LOW < SL """                    
        if df.high.loc[rownum] > c and df.low.loc[rownum] < b:
            
                    
                    
                    
                    
                    

In [ ]:
type(df.Capital.loc[rownum])

In [ ]:
df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]] = df[["TR", "N", "NPP", "FL", "SL", "Sep", "TrendDay", "FE_FL", "AE_FL", "SP", "En_price1", "Position", "a", "b", "c", "d", "En_price2", "Ex_price1", "Ex_price2", "Ex_price3", "Ex_price4", "Ex_price5", "P_L", "Capital"]].apply(pd.to_numeric)

In [ ]:
df.dtypes

In [ ]:
"""  
Iterates through rows after Counter1 = 1:  Assess each Pos=1,0,2 scenario against Volatility limits A,B,C,D  
                                            when necessary:
                                           - Set En_price1, En_price2
                                           - Set Ex_price1, Ex_price2, Ex_price3, Ex_price4
                                           - Update Counter(s)
                                           - Update Position(s)
                                           - Caculate & display P_L and change in Capital
                                           - move to next day (increment rownum)
"""

tick = 0.25
tick_val = 12.50
Capital = float(25000)

rownum = 50                                                 # start looking for uptrends from row 50


for rownum in range(50, 1499):
    
    if df.TrendDay.loc[rownum] >= 1 and df.TrendDir == "Up":
        
        
        """  CASE 1:  Position = 1 """
        if df.Position.loc[rownum] == 1:
            
            """ Identify Candle """
            if df.open.loc[rownum] > df.close.loc[rownum]:
                candle = "dark"
            elif df.open.loc[rownum] < df.close.loc[rownum]:
                candle = "open"
            elif df.open.loc[rownum] == df.close.loc[rownum]:
                candle_range = df.high.loc[rownum] - df.low.loc[rownum]
                if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
                    candle = "doji_open"
                else:
                    candle = "doji_dark"    # open & close occur in lower half of range            
                        
            if candle == "open":
                if df.low.loc[rownum] < b:        # SL_L: AE_FL add position AND stop loss on 2 positions
                    En_price2 = a - tick
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    Counter1 = ""
                    Counter2 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(En_price1 - Ex_price3) + (En_price1 - Ex_price4)] /tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < a:       # AP_L:  AE_FL add position and carry 2 to next day
                    En_price2 = a - tick
                    Counter1 += 1
                    Counter2 = 1
                    Position = 2
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)] /tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.high.loc[rownum] > c:      # TP_L: FE_FL take profit on 1 position and stage for Re-Entry
                    Ex_price1 = c + tick
                    Counter1 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    RE_CAN = True
                    df.P_L.loc[rownum] = (Ex_price1 - En_price1)/tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]  
                    rownum += 1

                elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
                    Counter1 += 1
                    df.Position.loc[rownum] = df.Position.loc[rownum - 1]
                    df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

            elif df.Position.loc[rownum - 1] == 1 and candle == "dark":

                if df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
                    Ex_price1 = c + tick
                    Counter1 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    RE_CAN = True
                    df.P_L.loc[rownum] = (Ex_price1 - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
                    En_price2 = a - tick
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    Counter1 = ""
                    Counter2 = ""
                    Position = 0
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(En_price1 - Ex_price3) + (En_price1 - Ex_price4)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
                    En_price2 = a - tick
                    Counter1 += 1
                    Counter2 = 1
                    Position = 2
                    df.Position.loc[rownum] = Position
                    df.P_L.loc[rownum] = [(df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1

                elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
                    Counter1 += 1
                    df.Position.loc[rownum] = df.Position.loc[rownum - 1]
                    df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
                    rownum += 1     
            else:
                rownum += 1
                    
                    
            
            
            
            
            
            
            
            
            
            

                
                
     
    
    
    
    elif df.TrendDay.loc[rownum] >= 1 and df.TrendDir.loc[rownum] == "Down":
        
        if df.Position.loc[rownum] == 1:
            
            """ Identify Candle """
            if df.open.loc[rownum] > df.close.loc[rownum]:
                candle = "dark"
            elif df.open.loc[rownum] < df.close.loc[rownum]:
                candle = "open"
            elif df.open.loc[rownum] == df.close.loc[rownum]:
                candle_range = df.high.loc[rownum] - df.low.loc[rownum]
                if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
                    candle = "doji_open"
                else:
                    candle = "doji_dark"    # open & close occur in lower half of range         
        
        
        
        Position = 0
        df.Position.loc[rownum] = 0
        SP = df.low.loc[rownum]
        df.SP.loc[rownum] = SP
        rownum += 1        

        while df.TrendDay.loc[rownum] > 0:
            if df.low.loc[rownum] < SP:
                En_price1 = SP - tick
                df.En_price1.loc[rownum] = En_price1
                Counter1 = 1
                df.Counter1.loc[rownum] = Counter1
                Position = 1
                df.Position.loc[rownum] = Position
                P_L = En_price1 - df.close.loc[rownum] 
                rownum += 1
                break
            else:
                SP = df.low.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1    

    else:
        rownum += 1
         

In [ ]:
    
"""  LONG Cases:  Position = 1, 0, and 2   """

rownum = 10

while df.TrendDay.loc[rownum] > 0:
    if df.open.loc[rownum] > df.close.loc[rownum]:
        candle = "dark"
    elif df.open.loc[rownum] < df.close.loc[rownum]:
        candle = "open"
    elif df.open.loc[rownum] == df.close.loc[rownum]:
        candle_range = df.high.loc[rownum] - df.low.loc[rownum]
        if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
            candle = "doji_open"
        else:
            candle = "doji_dark"    # open & close occur in lower half of range
    

                
# Case 1:  Position = 1: check for  AP, SL, TP, or ride   
        
if df.TrendDay.loc[rownum] >= 1:
    
    
    if df.Position.loc[rownum - 1] == 1 and candle == "open":
                
        if df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
            En_price2 = a - tick
            Ex_price3 = b - tick
            Ex_price4 = b - tick
            Counter1 = ""
            Counter2 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price1 - Ex_price4)
            df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
            rownum += 1
                        
        elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
            En_price2 = a - tick
            Counter1 += 1
            Counter2 = 1
            Position = 2
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
            Ex_price1 = c + tick
            Counter1 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            RE_CAN = True
            df.P_L.loc[rownum] = (Ex_price1 - En_price1)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]  
            rownum += 1

        elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
            Counter1 += 1
            df.Position.loc[rownum] = df.Position.loc[rownum - 1]
            df.P_L.loc[rownum] = df.close.loc[rownum] - En_price1
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

    elif df.Position.loc[rownum - 1] == 1 and candle == "dark":

        if df.high.loc[rownum] > c:      # FE_FL take profit on 1 position and stage for Re-Entry
            Ex_price1 = c + tick
            Counter1 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            RE_CAN = True
            df.P_L.loc[rownum] = (Ex_price1 - En_price1)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.low.loc[rownum] < b:        # AE_FL add position AND stop loss on 2 positions
            En_price2 = a - tick
            Ex_price3 = b - tick
            Ex_price4 = b - tick
            Counter1 = ""
            Counter2 = ""
            Position = 0
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price1 - Ex_price4)
            df.Capital.loc[rownum] = df.Capital.loc[rownum -1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.low.loc[rownum] < a:       # AE_FL add position and carry 2 to next day
            En_price2 = a - tick
            Counter1 += 1
            Counter2 = 1
            Position = 2
            df.Position.loc[rownum] = Position
            df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2)
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1

        elif df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:    # FE_FL < c and AE_FL > a ---> No action, ride trend
            Counter1 += 1
            df.Position.loc[rownum] = df.Position.loc[rownum - 1]
            df.P_L.loc[rownum] = df.close.loc[rownum] - En_price1
            df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + df.P_L.loc[rownum]
            rownum += 1     
    else:
        rownum += 1


In [ ]:
rownum = 1

while df.TrendDay.loc[rownum] >= 1:
    if df.open.loc[rownum] > df.close.loc[rownum]:
        candle = "dark"
    elif df.open.loc[rownum] < df.close.loc[rownum]:
        candle = "open"
    elif df.open.loc[rownum] == df.close.loc[rownum]:
        candle_range = df.high.loc[rownum] - df.low.loc[rownum]
        if df.open.loc[rownum] >= (df.low.loc[rownum] + candle_range/2):   # open & close occur at middle or in top half of range
            candle = "doji_open"
        else:
            candle = "doji_dark"    # open & close occur in lower half of range
                
                
                
                
                 # Case 2:  Position = 0: check for Re-Entry on two positions or ride zero
        
    if df.Position.loc[rownum - 1] == 0 and RE_CAN == True:
            
        """ RE_U: price at which to Re-enter on TWO positions """
        if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
            d = df.FL.loc[rownum - 1] + 1.2 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 30%N above FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
            d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
        elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
            d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
        else:
            d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

        df.d.loc[rownum] = d

        if df.low.loc[rownum] <= d:
            En_price1 = d - tick                        
            En_price2 = d - tick
            df.En_price1.loc[rownum] = En_price1
            df.En_price2.loc[rownum] = En_price2                        
            df.Counter1.loc[rownum] = 1
            df.Counter2.loc[rownum] = 1
            Position = 2
            df.Position.loc[rownum] = 2
            RE_CAN = False                              # toggle RE_CAN back to False
            df.RE_CAN.loc[rownum] = RE_CAN
            df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
            rownum += 1
            continue
        else:
            Position = 0
            df.Position.loc[rownum] = 0
            RE_CAN = True
            df.RE_CAN.loc[rownum] = True 
     
     
     
     
     


In [ ]:




                """ RE_U: price at which to Re-enter on TWO positions """
                if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
                    d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
                else:
                    d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

                df.d.loc[rownum] = d

                if df.low.loc[rownum] <= d:
                    En_price1 = d - tick                        
                    En_price2 = d - tick
                    df.En_price1.loc[rownum] = En_price1
                    df.En_price2.loc[rownum] = En_price2                        
                    df.Counter1.loc[rownum] = 1
                    df.Counter2.loc[rownum] = 1
                    Position = 2
                    df.Position.loc[rownum] = 2
                    RE_CAN = False                              # toggle RE_CAN back to False
                    df.RE_CAN.loc[rownum] = RE_CAN
                    df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
                    rownum += 1
                    continue
                else:
                    Position = 0
                    df.Position.loc[rownum] = 0
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = True 


In [ ]:
rownum = 10
init_cap = float(25000)
while df.TrendDay.loc[rownum] >= 1:
                
    """  C A S E S   Here:  Position = , High/Low reaction to A/B/C/D,  RE_CAN status     """
                
    # case 1:  ONE position - RM functions do not impact initial position so carry to next day
    if Position == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
        df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
        df.Position.loc[rownum] = 1
        df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) 
                    
        """ this conditional checks for same day Trend Change """
        if df.TrendDay.loc[rownum + 1] == 0:
            Counter1 = ""
            df.Counter1.loc[rownum + 1] = ""
            Position = 0
            df.Position.loc[rownum + 1] = ""
            Ex_price5 = df.close.loc[rownum]
            df.Ex_price5.loc[rownum] = Ex_price5
            df.P_L.loc[rownum] = En_price1 - Ex_price5
        else:
            rownum += 1
            continue                        
            
            
            

In [ ]:
type(init_cap)

In [ ]:


addpos = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},                            # syntax to call, enter script as follows:
          "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}                             # addpos["L"]["2"]  yields: 0.015
          }                                                                        # stoploss["S"]["3"]  yields: 0.0195

stoploss = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

takeprof = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

re_entry = {"L" : {"1" : 1.10, "2" : 1.50, "3" : 1.95},
            "S" : {"1" : 1.10, "2" : 1.50, "3" : 1.95}
            }

X = 0.25   # General Risk Mgmt. multiplier

tick = 0.25
tick = float(tick)
tick_val = 12.50
tick_val = float(tick_val)


for rownum in range(1, 1500):
    Position = 0
    if df.TrendDay.loc[rownum] == 0 and df.TrendDir.loc[rownum] == "Up":
        SP = df.high.loc[rownum]                            # assign the 'high' slice to the SP variable
        df.SP.loc[rownum] = SP                              # equate the SP slice with the SP variable
        rownum += 1

        while df.TrendDay.loc[rownum] >= 1 and Position == 0:    # this 'while' loop finds initial entry, records initial P_L, then advances to next rownum

            if df.high.loc[rownum] > SP: 
                En_price1 = SP + tick
                df.En_price1.loc[rownum] = En_price1
                df.Counter1.loc[rownum] = 1                  # initialize Counter1
                Position = 1
                df.Position.loc[rownum] = 1                  # 'position' variable
                # df.Position.loc[rownum] = Position         # this [rownum] dataframe slice equal to this [rownum] value for position variable
                df.P_L.loc[rownum] = ( (df.close.loc[rownum] - En_price1)) 
                rownum += 1
                break

            else:
                SP = df.high.loc[rownum]
                df.SP.loc[rownum] = SP
                rownum += 1                

        while df.TrendDay.loc[rownum] > 1:

            """ AP_U function: AE_FL: price at which to Add to initial position """
            if df.NPP.loc[rownum - 1] < addpos["L"]["1"]:
                a = df.FL.loc[rownum - 1] +  X * df.N.loc[rownum - 1]
            elif df.NPP.loc[rownum - 1] <= addpos["L"]["2"]:                    
                a = df.FL.loc[rownum - 1]
            elif df.NPP.loc[rownum - 1] <= addpos["L"]["3"]:
                a = df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]
            else:
                a = df.FL.loc[rownum - 1] - 6.4 * X * df.N.loc[rownum - 1]
                
            df.a.loc[rownum] = a   # print value in dataframe slice 

            """ SL_U function: AE_FL: price at which to cut losses """
            if df.NPP.loc[rownum - 1] <= stoploss["L"]["1"]:
                b =  df.FL.loc[rownum - 1] - 5 * X * df.N.loc[rownum - 1]     # stop loss when price retraces 1N below FL
            elif df.NPP.loc[rownum - 1] <= stoploss["L"]["2"]:
                b = df.FL.loc[rownum - 1] - 6 * X * df.N.loc[rownum - 1]  # stop loss when price retraces 150%N below FL
            elif df.NPP.loc[rownum - 1] <= stoploss["L"]["3"]:
                b = df.FL.loc[rownum - 1] - 7 * X * df.N.loc[rownum - 1]   # stop loss when price retraces  175%N below FL
            else:
                b =  df.FL.loc[rownum - 1] - 8 * X * df.N.loc[rownum - 1]

            df.b.loc[rownum] = b

            """ TP_U function: FE_FL: price at which to take profits """
            if df.NPP.loc[rownum - 1] <= takeprof["L"]["1"]:
                c = df.FL.loc[rownum - 1] + 5 * X * df.N.loc[rownum - 1]   # for NPP < 1.10%: take profit when price exceeds 125%N above previous FL
            elif df.NPP.loc[rownum - 1] <= takeprof["L"]["2"]:
                c = df.FL.loc[rownum - 1] + 5.6 * X * df.N.loc[rownum - 1]    # for NPP < 1.50%: take profit when price exceeds 140%N (5.6 * X) above previous FL
            elif df.NPP.loc[rownum - 1] <= takeprof["L"]["3"]:
                c = df.FL.loc[rownum - 1] + 7 * X * df.N.loc[rownum - 1]    # for NPP < 1.95%: take profit when price exceeds 175%N (7*X*N) above previous FL
            else:
                c = df.FL.loc[rownum - 1] + 7.5 * X * df.N.loc[rownum - 1]                    

            df.c.loc[rownum] = c

            # case 1:  ONE position - RM functions do not impact initial position so carry to next day
            if Position == 1 and df.high.loc[rownum] <=c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Position.loc[rownum] = 1
                df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) 
                    
                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue            

            # case 2:  ONE position - FE_FL take profit on initial position   
            elif Position == 1 and df.high.loc[rownum] > c:
                Ex_price1 = c + tick
                df.Ex_price1.loc[rownum] = Ex_price1
                df.Counter1.loc[rownum] = 0
                Position = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] = (Ex_price1 - En_price1)

                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue                    

            # case 3:  FLAT due to FE_FL triggering TP - look for AE_FL retracement to Re-enter TWO positions
            elif Position == 0 and df.RE_CAN.loc[rownum] == True:

                """ RE_U: price at which to Re-enter on TWO positions """
                if df.NPP.loc[rownum - 1] <= re_entry["L"]["1"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["2"]:
                    d = df.FL.loc[rownum - 1] + 0.5 * X * df.N.loc[rownum - 1]   # Re-enter when price retraces within 12.5%N above FL
                elif df.NPP.loc[rownum - 1] <= re_entry["L"]["3"]:
                    d = df.FL.loc[rownum - 1] - 2 * X * df.N.loc[rownum - 1]     # Re-enter when price retraces below 50%N below FL
                else:
                    d = df.FL.loc[rownum - 1] - 2.25 * X * df.N.loc[rownum - 1]                        

                df.d.loc[rownum] = d

                if df.low.loc[rownum] <= d:
                    En_price1 = d - tick                        
                    En_price2 = d - tick
                    df.En_price1.loc[rownum] = En_price1
                    df.En_price2.loc[rownum] = En_price2                        
                    df.Counter1.loc[rownum] = 1
                    df.Counter2.loc[rownum] = 1
                    Position = 2
                    df.Position.loc[rownum] = 2
                    RE_CAN = False                              # toggle RE_CAN back to False
                    df.RE_CAN.loc[rownum] = RE_CAN
                    df.P_L.loc[rownum] = ((df.close.loc[rownum] - En_price1 + df.close.loc[rownum] - En_price2))                    
                    rownum += 1
                    continue
                else:
                    Position = 0
                    df.Position.loc[rownum] = 0
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = True 

                    """ this conditional checks for same day Trend Change """
                    if df.TrendDay.loc[rownum + 1] == 0:
                        Counter1 = ""
                        df.Counter1.loc[rownum + 1] = ""
                        Position = 0
                        df.Position.loc[rownum + 1] = ""
                        Ex_price5 = df.close.loc[rownum]
                        df.Ex_price5.loc[rownum] = Ex_price5
                        df.P_L.loc[rownum] = En_price1 - Ex_price5
                    else:
                        rownum += 1
                        continue             

            # case 4:  TWO positions - FE_FL take profit on two positions
            elif Position == 2 and df.high.loc[rownum] > c:
                Ex_price1 = c + tick
                Ex_price2 = c + tick
                df.Counter1.loc[rownum] = 0
                df.Counter2.loc[rownum] = 0
                Position = 0
                df.Position.loc[rownum] = 0
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                df.P_L.loc[rownum] =  ( ((Ex_price1 - En_price1) + (Ex_price2 - En_price2)) ) 
                rownum += 1
                continue

            # case 5: TWO positions - RM functions do no impact position with contracts so carry to next rownum
            elif Position == 2 and df.high.loc[rownum] <= c and df.low.loc[rownum] >= a:
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = df.Counter2.loc[rownum-1] + 1
                Position = 2
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (((df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2))) 

                """ this conditional checks for same day Trend Change """
                if df.TrendDay.loc[rownum + 1] == 0:
                    Counter1 = ""
                    df.Counter1.loc[rownum + 1] = ""
                    Position = 0
                    df.Position.loc[rownum + 1] = ""
                    Ex_price5 = df.close.loc[rownum]
                    df.Ex_price5.loc[rownum] = Ex_price5
                    df.P_L.loc[rownum] = En_price1 - Ex_price5
                else:
                    rownum += 1
                    continue                    


            # case 6a: ONE position - AE_FL - add to initial position and STOP out of TWO positions - look for next trend start
            elif Position == 1 and df.low.loc[rownum] < a:
                En_price2 = a - tick
                df.En_price2.loc[rownum] = En_price2
                df.Counter1.loc[rownum] = df.Counter1.loc[rownum-1] + 1
                df.Counter2.loc[rownum] = 1
                Position = 2
                df.Position.loc[rownum] = 2
                df.P_L.loc[rownum] = (df.close.loc[rownum] - En_price1) + (df.close.loc[rownum] - En_price2) 


                # case 6b: TWO positions - AE_FL - STOP OUT both positions and increment until next TrendDay=0
                if df.low.loc[rownum] < b:
                    Ex_price3 = b - tick
                    Ex_price4 = b - tick
                    df.Ex_price3.loc[rownum] = Ex_price3
                    df.Ex_price4.loc[rownum] = Ex_price4               
                    df.Counter1.loc[rownum] = 0
                    df.Counter2.loc[rownum] = 0
                    Position = 0
                    df.Position.loc[rownum] = 0 
                    df.P_L.loc[rownum] = (En_price1 - Ex_price3) + (En_price2 - Ex_price4) 
                    rownum += 1
                    while df.TrendDay.loc[rownum] != 0:
                        rownum += 1
                        continue
                    break
                else:
                    rownum += 1
                    continue
        else:
            rownum += 1
    else:
        rownum += 1        

In [ ]:
rownum = 10
for rownum in range(10, 1499):

    """ Position = 1 Case 1: High > TP and LOW < AP  """  

    if df.Position.loc[rownum] = 1: 

        if df.high.loc[rownum] > c and df.low.loc[rownum] < a:

            if df.close.loc[rownum] > df.open.loc[rownum]:  # (candle=OPEN): low occurs before high.  Add position then take profit on both positions.  Set Re-Entry to True
                # Pos 1 ---> Pos 2
                # Pos 2 ---> Pos 0

                Position = 0
                df.Position.loc[rownum] = Position
                En_price2 = a - tick
                Ex_price1 = c + tick
                Ex_price2 = c + tick
                Counter1 = ""
                Counter2 = ""
                RE_Can = True
                df.RE_CAN.loc[rownum] = RE_CAN
                P_L = [(Ex_price1 - En_price1) + (Ex_price2 - En_price2)] / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + P_L
                rownum += 1

            elif df.close.loc[rownum] < df.open.loc[rownum]:   # (candle=DARK): high occurs before low.  Take profit on initial position.  Set Re-Entry to True.
                # Pos 1 ---> Pos 0

                Position = 0
                df.Position.loc[rownum] = Position
                Counter1 = ""
                Ex_price1 = c + tick
                RE_CAN = True
                df.RE_CAN.loc[rownum] = RE_CAN
                P_L = [(df.close.loc[rownum] - Ex_price1)] / tick * tick_val
                df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + P_L
                rownum += 1


            elif df.close.loc[rownum] = df.open.loc[rownum]:
                Candle_Range = df.high.loc[rownum] - df.low.loc[rownum]
                if df.open.loc[rownum] >= (df.low.loc[rownum] +  0.5 * Candle_Range):  # open & close occur at middle or in top half of range
                    candle = "doji_open"  # treat like candle=OPEN
                    Position = 0
                    df.Position.loc[rownum] = Position
                    En_price2 = a - tick
                    Ex_price1 = c + tick
                    Ex_price2 = c + tick
                    Counter1 = ""
                    Counter2 = ""
                    RE_CAn = True
                    df.RE_CAN.loc[rownum] = RE_CAN
                    P_L = [(Ex_price1 - En_price1) + (Ex_price2 - En_price2)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + P_L
                    rownum += 1


                elif df.open.loc[rownum] < (df.low.loc[rownum] + 0.5 * Candle_Range):
                    candle = "doji_dark"   # open & close occur in lower half of range:  treat like candle=DARK
                    Position = 0
                    Counter1 = ""
                    Ex_price1 = c + tick
                    RE_CAN = True
                    df.RE_CAN.loc[rownum] = RE_CAN
                    P_L = (Close - Ex_price1)] / tick * tick_val
                    df.Capital.loc[rownum] = df.Capital.loc[rownum - 1] + P_L
                    rownum += 1



 




""" Case 2: High > TP and LOW < SL 

if 



"""

""" Case 3: High < TP and Low < AP """

""" Case 4: High < TP and Low > SL """

""" Case 5: High < TP and Low > AP """